In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [9]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [0]
  ng_refs = [7]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(600, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(2, 1.6, 1.6, 1.6, 1.6, 1.6, 1.8)]
  wgt = [(0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.9)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.001259                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.000295
0.000750      2.0 -0.001142                  2.0  0.000246
0.001052      3.0 -0.000861                  3.0  0.000605
0.001476      4.0 -0.000511                  4.0  0.000836
0.002070      5.0 -0.000086                  5.0  0.001098
0.002904      6.0  0.000426                  6.0  0.001402
0.004074      7.0  0.001038                  7.0  0.001759
0.005714      8.0  0.001761                  8.0  0.002176
0.008015      9.0  0.002612                  9.0  0.002656
0.011243     10.0  0.003607                 10.0  0.003206
0.015771     11.0  0.004761                 11.0  0.003835
0.022122     12.0  0.006094                 12.0  0.004553
0.031031     13.0  0.007617                 13.0  0.005371
0.043528     14.0  0.009131                 14.0  0.006185
0.061057     15.0  0.010389                 15.0  0.006876
0.085645     16.0  0.011526                 16.0  0.007523
0.120136     17.0  0.012713                 17.0  0.008216
0.168516     18.0  0.013951                 18.0  0.008957
0.236378     19.0  0.015242                 19.0  0.009749
0.331549     20.0  0.016599                 20.0  0.010598
0.465100     21.0  0.017884                 21.0  0.011431
0.652400     22.0  0.016329                 22.0  0.010827
0.915100     23.0  0.011701                 23.0  0.008513
1.283650     24.0  0.007515                 24.0  0.006221
1.800600     25.0  0.004612                 25.0  0.004454
2.525700     26.0  0.002662                 26.0  0.003110
3.542800     27.0  0.001392                 27.0  0.002099
4.969550     28.0  0.000710                 28.0  0.001435
6.970850     29.0  0.000442                 29.0  0.001055
9.778100     30.0  0.000315                 30.0  0.000800
13.715850    31.0  0.000212                 31.0  0.000591
19.239350    32.0  0.000125                 32.0  0.000427
26.987250    33.0  0.000127                 33.0  0.000367
37.855300    34.0  0.000218                 34.0  0.000405
53.100050    35.0  0.000308                 35.0  0.000465
73.887500    36.0  0.000385                 36.0  0.000518
97.662500    37.0  0.000437                 37.0  0.000540
121.437500   38.0  0.000435                 38.0  0.000510
145.212500   39.0  0.000533                 39.0  0.000598
168.987500   40.0  0.000586                 40.0  0.000617
192.762500   41.0  0.000588                 41.0  0.000564
216.537500   42.0  0.000490                 42.0  0.000406
240.312500   43.0  0.000320                 43.0  0.000193
264.087500   44.0  0.000055                 44.0 -0.000093
287.862500   45.0 -0.000336                 45.0 -0.000471
311.637500   46.0 -0.000471                 46.0 -0.000569
335.412500   47.0  0.000097                 47.0  0.000036
359.187500   48.0  0.001223                 48.0  0.001162
382.962500   49.0  0.003170                 49.0  0.003033
406.737500   50.0  0.006179                 50.0  0.005956
430.512500   51.0  0.009862                 51.0  0.009623
454.287500   52.0  0.014103                 52.0  0.013596
478.062500   53.0  0.018802                 53.0  0.017768
501.837500   54.0  0.023892                 54.0  0.022588
525.612500   55.0  0.029291                 55.0  0.028268
549.387500   56.0  0.034865                 56.0  0.034232
573.162500   57.0  0.040476                 57.0  0.039771
596.937500   58.0  0.045917                 58.0  0.044643
620.712500   59.0  0.050988                 59.0  0.049030
644.487500   60.0  0.055502                 60.0  0.053312
668.262500   61.0  0.059334                 61.0  0.057437
692.037500   62.0  0.062449                 62.0  0.061367
715.812500   63.0  0.064898                 63.0  0.064722
739.587500   64.0  0.066843                 64.0  0.067413
76

# Fluxes by Layer

CRD                            CLIRAD                \
                        flug          fldg     fnetg      flug          fldg   
pressure     level                                                             
0.000000e+00 1     -8.627833  0.000000e+00 -8.627833       NaN           NaN   
1.000000e-08 1           NaN           NaN       NaN -8.687292  9.417060e-08   
6.244000e-04 2     -8.627833  9.079452e-08 -8.627833 -8.687292  1.438810e-07   
8.759000e-04 3     -8.627833  1.303413e-07 -8.627833 -8.687292  1.705074e-07   
1.228600e-03 4     -8.627833  1.952246e-07 -8.627833 -8.687292  2.166009e-07   
1.723400e-03 5     -8.627834  3.017399e-07 -8.627833 -8.687292  2.858755e-07   
2.417400e-03 6     -8.627834  4.762605e-07 -8.627833 -8.687292  3.918064e-07   
3.390900e-03 7     -8.627834  7.617428e-07 -8.627833 -8.687292  5.562304e-07   
4.756500e-03 8     -8.627834  1.227917e-06 -8.627833 -8.687292  8.154279e-07   
6.672000e-03 9     -8.627835  1.987182e-06 -8.627833 -8.687292  1.228041e-06   
9.358900e-03 10    -8.627835  3.221389e-06 -8.627832 -8.687292  1.887272e-06   
1.312780e-02 11    -8.627835  5.223075e-06 -8.627830 -8.687291  2.942274e-06   
1.841450e-02 12    -8.627836  8.460646e-06 -8.627827 -8.687291  4.631719e-06   
2.583020e-02 13    -8.627835  1.368225e-05 -8.627822 -8.687289  7.337905e-06   
3.623230e-02 14    -8.627835  2.207239e-05 -8.627812 -8.687287  1.166886e-05   
5.082340e-02 15    -8.627832  3.528800e-05 -8.627797 -8.687283  1.848661e-05   
7.129060e-02 16    -8.627827  5.544345e-05 -8.627771 -8.687277  2.893441e-05   
1.000000e-01 17    -8.627818  8.566377e-05 -8.627732 -8.687267  4.475216e-05   
1.402710e-01 18    -8.627802  1.307462e-04 -8.627672 -8.687252  6.868476e-05   
1.967600e-01 19    -8.627776  1.976102e-04 -8.627578 -8.687228  1.048621e-04   
2.759970e-01 20    -8.627731  2.960771e-04 -8.627435 -8.687191  1.594986e-04   
3.871000e-01 21    -8.627657  4.397814e-04 -8.627217 -8.687134  2.419234e-04   
5.431000e-01 22    -8.627532  6.458637e-04 -8.626886 -8.687047  3.656229e-04   
7.617000e-01 23    -8.627362  8.983573e-04 -8.626463 -8.686931  5.305932e-04   
1.068500e+00 24    -8.627183  1.144871e-03 -8.626038 -8.686809  7.175766e-04   
1.498800e+00 25    -8.627015  1.359903e-03 -8.625655 -8.686692  9.174429e-04   
2.102400e+00 26    -8.626863  1.537780e-03 -8.625325 -8.686585  1.129523e-03   
2.949000e+00 27    -8.626734  1.675964e-03 -8.625058 -8.686497  1.353067e-03   
4.136600e+00 28    -8.626639  1.776424e-03 -8.624863 -8.686436  1.587461e-03   
5.802500e+00 29    -8.626579  1.857100e-03 -8.624722 -8.686407  1.841887e-03   
8.139200e+00 30    -8.626549  1.949214e-03 -8.624600 -8.686412  2.138404e-03   
1.141700e+01 31    -8.626553  2.075708e-03 -8.624477 -8.686460  2.497395e-03   
1.601470e+01 32    -8.626616  2.254162e-03 -8.624362 -8.686577  2.936039e-03   
2.246400e+01 33    -8.626780  2.514395e-03 -8.624266 -8.686795  3.481381e-03   
3.151050e+01 34    -8.627077  2.946976e-03 -8.624130 -8.687139  4.217372e-03   
4.420010e+01 35    -8.627511  3.709424e-03 -8.623802 -8.687613  5.300977e-03   
6.200000e+01 36    -8.628121  4.969041e-03 -8.623152 -8.688255  6.924529e-03   
8.577500e+01 37    -8.628920  6.854286e-03 -8.622066 -8.689083  9.210343e-03   
1.095500e+02 38    -8.629707  8.872782e-03 -8.620834 -8.689893  1.153988e-02   
1.333250e+02 39    -8.630541  1.093279e-02 -8.619608 -8.690747  1.383093e-02   
1.571000e+02 40    -8.631816  1.370774e-02 -8.618108 -8.692035  1.680232e-02   
1.808750e+02 41    -8.633662  1.720440e-02 -8.616457 -8.693873  2.037904e-02   
2.046500e+02 42    -8.636266  2.146564e-02 -8.614800 -8.696436  2.453035e-02   
2.284250e+02 43    -8.639641  2.622002e-02 -8.613421 -8.699739  2.897637e-02   
2.522000e+02 44    -8.643941  3.142145e-02 -8.612520 -8.703931  3.371117e-02   
2.759750e+02 45    -8.649511  3.714635e-02 -8.612365 -8.709353  3.887024e-02   
2.997500e+02 46    -8.656867  4.355673e-02 -8.613310 -8.716497  4.468634e-02   
3.235250e+02 47    -8.666

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')